# wine shop system

In [ ]:
import requests 
import pandas as pd
import json

cart = {
    'Delivery Address': {},
    'Total Amount (R$)': 0,
    'Purchased Wines': {}
}

wines = {
    'type': ['red', 'rosé', 'dry', 'white', 'sweet', 'sparkling', 'dessert', 'fortified', 'organic', 'biodynamic', 'natural'],
    '% alcohol': [11.0, 15.0, 12.0, 13.0, 10.0, 12.5, 13.5, 20.0, 12.0, 12.0, 11.5],
    'vintage': [1958, 1962, 1970, 1994, 2002, 2010, 2005, 1985, 2018, 2019, 2020],
    'price': [100.0, 130.0, 20.0, 25.0, 50.0, 150.0, 200.0, 300.0, 75.0, 85.0, 90.0],
    'Nationality': ['Chilean', 'Argentinian', 'French', 'Italian', 'Brazilian', 'Spanish', 'Portuguese', 'Australian', 'South African', 'New Zealand', 'German'],
    'Stock': [1, 2, 3, 4, 5, 6, 2, 1, 5, 4, 3]
}

#wine_list = [str(i) for i in range(len(wines['type']))]

types = {key : type(wines[key][0]) for key in wines.keys()}
#print(types)
wine_list = []
for i in range(len(wines['type'])):
    wine_list.append(str(i))
    
def force_user(prompt,list_options,Max_attempts = None):
    user_prompt = input(prompt)
    attempts = 0
    while user_prompt not in list_options:
        print(f'Attempt {attempts}: Please enter a valid answer')
        user_prompt = input(prompt)
        if Max_attempts:
            if attempts>=Max_attempts-1:
                print("You have exceeded the max number of attempts")
                return False
            attempts +=1
    return user_prompt

def print_dictionary(dic):
    for key in dic.keys():
        if dic[key] is dict:
            print_dictionary(dic[key])
        else:
            print(f'{key} - {dic[key]}')
    return


def postal_code():
    while True:
        try:
            user_postal_code = input("Enter your postal code (international format): ")

            if user_postal_code.isdigit() and len(user_postal_code) == 8:
                url = f"https://viacep.com.br/ws/{user_postal_code}/json/"
                response = requests.get(url)
            else:
                url = f"http://zipcloud.ibsnet.co.jp/api/search?zipcode={user_postal_code}"
                response = requests.get(url)

            if response.status_code == 200:
                response = response.json()
                if "erro" in response:
                    print("Invalid postal code!")
                    continue
                if 'cep' in response:
                    print_dictionary(response)
                else:
                    try:
                        address = response['results'][0]
                        response = {
                            "postal_code": address['zipcode'],
                            "street": address['address1'] + address['address2'] + address['address3'],
                            "complement": "",
                            "neighborhood": "",
                            "city": address['address1'],
                            "state": "",
                            "unit": "",
                            "ibge": "",
                            "gia": ""
                        }
                        print_dictionary(response)
                    except (IndexError, KeyError, TypeError) as e:
                        print(f"Error processing address data: {e}")
                        continue

                correct = force_user("Is the information correct? (y/n): ", ['y', 'n'],5)
                if correct == 'y':
                    number = input("Enter your house number: ")
                    complement = input("Enter the complement: ")
                    response['number'] = number
                    response['complement'] = complement
                    cart['Delivery Address'] = response
                    return
            else:
                print("Invalid postal code!")
        except requests.RequestException as e:
            print(f"Network error: {e}")
        except ValueError as e:
            print(f"Invalid input: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            
    return

def login():
    username = force_user('type your username',['ayumi'],5)
    if username:
        password = force_user('type your password',['yamazaki'],5)
    else:
        return False
    print('succeded  login')
    return username and password

def remove_wine():
    display(pd.DataFrame(wines))
    remove_index =  int(force_user('tell wich wine do you want to remove',wine_list,8))
    for key in wines.keys():
        wines[key].pop(remove_index)
    display(pd.DataFrame(wines))
    return

def register_wine():
    display(pd.DataFrame(wines))
    for key in wines.keys():
        if type(wines[key][0]) is str:
            info = input(f'tell me the {key}: ')
            wines[key].append(info)
        elif type(wines[key][0]) is float:
            while True:
                try:
                    info = float(input(f'tell me the {key}: '))
                except ValueError:
                    print('the value must be a float. For example: 2.3')
                else:
                    wines[key].append(info)
                    break
        elif type(wines[key][0]) is int:
            while True:
                try:
                    info = int(input(f'type the new {key}: '))
                except ValueError:
                    print('the number must be int value for example: 4')
                else:
                    wines[key].append(info)
                    break
    display(pd.DataFrame(wines))
    return


def change_price():
    display(pd.DataFrame(wines))
    wine = int(force_user('type which wine you want to change the price', wine_list,8))
    while True:
        try:
            new_price = float(input("Type the new price: "))
        except ValueError:
            print("You must type a valid number, e.g., 99.90")
        else:
            wines['price'][wine] = new_price
            print("Succeeded in changing price")
            break
    display(pd.DataFrame(wines))
    return
        

start = force_user("welcome to the winery, you're costumer yor employee? (c/e)",['c','e'])
if start == 'c':     
    postal_code()
    while True:
        print('Essas sÃ£o nossas opÃ§Ãµes de vinhos:')
        display(pd.DataFrame(wines))
        option = int(force_user("wich one you liked the most? ",wine_list))
        display(pd.DataFrame(wines))
        for key in wines.keys():
            print(f"{key} : {wines[key][option]}")
        user_choice = force_user("do you want to bring the wine (y/n)?",['y','n'],5)
        if user_choice == 'y':
            if wines['Stock'][option] == 0:
                print(f"Sorry, we no longer have {wines['type'][option]}, go back to the menu!")
                continue
            else:
                wines['Stock'][option] -= 1
            cart['Total Amount (R$)'] += wines['price'][option]
            wine_chosen = wines['type'][option]
            if wine_chosen not in cart['Purchased Wines'].keys():
                cart['Purchased Wines'][wine_chosen] = 1
            else:
                cart['Purchased Wines'][wine_chosen] += 1
        continue_ = force_user("Would you like to see more wines (y/n)? ", ['y','n'],5)
        if continue_ == 'n':
            print("thanks for shooping with us")
            print('-'*60)
            print(print_dictionary(cart))
            display(pd.DataFrame(wines))
            break
else:
    login()
    if login:
        while True:
            option = force_user(f"Which operation do you want to perform? \n1-Change pre-order \
                                \n2-Register wine\n" f"3-Remove wine\n4-Exit\n->",['1','2','3','4'])
            if option == '1':
                change_price()
                print(pd.DataFrame(wines))
            elif option == '2':
                register_wine()
                wine_list = []
                for i in range(len(wines['type'])):
                    wine_list.append(str(i))
                #print(wine_list)
                print(pd.DataFrame(wines))
            elif option == '3':
                remove_wine()
                wine_list = []
                for i in range(len(wines['type'])):
                    wine_list.append(str(i))
                #print(wine_list)
                print(pd.DataFrame(wines))
            else:
                print(wine_list)
                break 

            
    


welcome to the winery, you're costumer yor employee? (c/e)e
type your usernameayumi
type your passwordyamazaki
succeded  login
Which operation do you want to perform? 
1-Change pre-order                                 
2-Register wine
3-Remove wine
4-Exit
->2


,type,% alcohol,vintage,price,Nationality,Stock
0,red,11.0,1958,100.0,Chilean,1
1,rosé,15.0,1962,130.0,Argentinian,2
2,dry,12.0,1970,20.0,French,3
3,white,13.0,1994,25.0,Italian,4
4,sweet,10.0,2002,50.0,Brazilian,5
5,sparkling,12.5,2010,150.0,Spanish,6
6,dessert,13.5,2005,200.0,Portuguese,2
7,fortified,20.0,1985,300.0,Australian,1
8,organic,12.0,2018,75.0,South African,5
9,biodynamic,12.0,2019,85.0,New Zealand,4


tell me the type: 3
tell me the % alcohol: 2345
type the new vintage: 11
tell me the price: 78
tell me the Nationality: 
type the new Stock: 6


,type,% alcohol,vintage,price,Nationality,Stock
0,red,11.0,1958,100.0,Chilean,1
1,rosé,15.0,1962,130.0,Argentinian,2
2,dry,12.0,1970,20.0,French,3
3,white,13.0,1994,25.0,Italian,4
4,sweet,10.0,2002,50.0,Brazilian,5
5,sparkling,12.5,2010,150.0,Spanish,6
6,dessert,13.5,2005,200.0,Portuguese,2
7,fortified,20.0,1985,300.0,Australian,1
8,organic,12.0,2018,75.0,South African,5
9,biodynamic,12.0,2019,85.0,New Zealand,4


          type  % alcohol  vintage  price    Nationality  Stock
0          red       11.0     1958  100.0        Chilean      1
1         rosé       15.0     1962  130.0    Argentinian      2
2          dry       12.0     1970   20.0         French      3
3        white       13.0     1994   25.0        Italian      4
4        sweet       10.0     2002   50.0      Brazilian      5
5    sparkling       12.5     2010  150.0        Spanish      6
6      dessert       13.5     2005  200.0     Portuguese      2
7    fortified       20.0     1985  300.0     Australian      1
8      organic       12.0     2018   75.0  South African      5
9   biodynamic       12.0     2019   85.0    New Zealand      4
10     natural       11.5     2020   90.0         German      3
11           3     2345.0       11   78.0                     6
Which operation do you want to perform? 
1-Change pre-order                                 
2-Register wine
3-Remove wine
4-Exit
->4
['0', '1', '2', '3', '4', '5', '6'